## Imports

In [28]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium

In [24]:
df = pd.read_csv('HFZ_purchaed_properties.csv', dtype='str')

In [25]:
df['Full_address'] = df['Index'] + ' ' + df['#'] + ' ' + df['(PIN)'] + ' ' + df['Address']

In [29]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [31]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['Full_address'].apply(geocode)

In [32]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

In [47]:
df.columns

Index(['Unnamed: 0', 'Property', 'Index', '#', '(PIN)', 'Address', 'PropType',
       'Unit', 'S', 'T', 'R', 'Subdiv', 'Lot', 'Block', 'Part', 'Lot.1',
       'Bldg', 'Full_address', 'geocoded', 'lat', 'lon'],
      dtype='object')

In [60]:
df2 = df.loc[df['Full_address'] == '7507 N CLARK ST']
df2 = df2.head(1)

In [36]:
def popup_html(row):
    address = row['Full_address']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    </html>
    '''
    return html

In [62]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df2[["lat", "lon"]].mean().to_list(), zoom_start=13)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Ashkenazy purchase in Rogers Park')


### Create map container ###
m = folium.Map(location=df2[["lat", "lon"]].mean().to_list(),zoom_start=16,tiles=None)

for index, row in df2.iterrows():
    marker = folium.Marker(
        location=[row['lat'], row['lon']],
        radius=5,
        fill=True,
        popup=folium.Popup(popup_html(row), max_width=400))
    marker.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
            
# Display map
m

In [63]:
m.save('index.html')

## Map URL Snagger

In [64]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [65]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/CHI_HFZ_purchase
